>> # Modelo preditivo -  Aguas do Brasil
- Modelagem preditiva para  Devedores e CPC com base em título e títulos pagos e informações do telefone
- Este Jupyter contém as etapas de ETL e Modelagem

# Importando pacotes e bibliotecas

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os , gc, time, urllib, pyodbc, warnings
warnings.filterwarnings("ignore")
from datetime import date, timedelta
import sqlalchemy as sql
from sqlalchemy import create_engine

# ETL


In [2]:
params = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                 "SERVER=10.10.220.100;"
                                 "DATABASE=dbActyon_GAB;" # trocar o nome da operação
                                 "UID=ralisson.local;"
                                 "PWD=A4TIL&fRplPn;"
                                 "Trusted_Connection=no")
engine = sql.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
engine.connect

<bound method Engine.connect of Engine(mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BSQL+Server+Native+Client+11.0%7D%3BSERVER%3D10.10.220.100%3BDATABASE%3DdbActyon_GAB%3BUID%3Dralisson.local%3BPWD%3DA4TIL%26fRplPn%3BTrusted_Connection%3Dno)>

In [3]:
#Criando variavei de datas para coleta da query sql

TODAY = date.today()
YESTERDAY = date.today() - timedelta(days=200)
data_inicio = f"{YESTERDAY.year}-{YESTERDAY.month}-{YESTERDAY.day}" #yesterday 
data_final = f"{TODAY.year}-{TODAY.month}-{TODAY.day}"

In [ ]:
#Importando dados do banco de dados SQL

query = '''

    SELECT distinct
        tbtitulo.TITULO_ID,
        tbdevedor.CEP,
        tbdevedor_fone.ORIGEM,
        tbdevedor_fone.TIPO,
        tbdevedor.QTDE_TITULOS,
        tbdevedor.VALOR_DIVIDA_ATIVA,
        tbtitulo_contrato_indicador.INDICADOR,
        tbtitulo_pago.TIPO_BAIXA

    FROM tbtitulo  (NOLOCK)
        JOIN tbtitulo_pago  (NOLOCK) ON tbtitulo.TITULO_ID = tbtitulo_pago.TITULO_ID
        JOIN tbdevedor  (NOLOCK) ON tbdevedor.DEVEDOR_ID = tbtitulo.DEVEDOR_ID
        JOIN tbdevedor_fone  (NOLOCK) ON tbdevedor.DEVEDOR_ID = tbdevedor_fone.DEVEDOR_ID
        JOIN tbtitulo_contrato_indicador  (NOLOCK) ON tbdevedor.DEVEDOR_ID = tbtitulo_contrato_indicador.DEVEDOR_ID

    WHERE tbdevedor_fone.STATUS = 0
    AND tbdevedor.DATA_IMPORTACAO between
    

'''
#df = pd.read_sql(query, engine)
df = pd.read_sql(query + f"'{data_inicio}' AND '{data_final}'", engine)
print("tamanho da tabela: ", df.shape)

engine.dispose()
gc.collect()

In [ ]:
df = df.drop_duplicates()
#df.dropna(inplace = True)
df.shape

# Engenharia de Atributos

In [ ]:
#Trabalhando com datas
TODAY = date.today()
TODAY = pd.to_datetime(TODAY)

In [ ]:
#alterando tipo das featrures
df['TIPO_BAIXA'] = df['TIPO_BAIXA'].astype('category')
df['CEP'] = df['CEP'].astype('category')
df['TITULO_ID'] = df['TITULO_ID'].astype(str)
df['ORIGEM'] = df['ORIGEM'].astype('category')
df['INDICADOR'] = df['INDICADOR'].astype('category')
df['TIPO'] = df['TIPO'].astype('category')

In [ ]:
df.TIPO_BAIXA.value_counts()

In [ ]:
#Variavel alvo Pagamento de títulos
df['TARGET']  = df['TIPO_BAIXA'].replace(['1', '2' , '4','5', '3', '8','7',None, '0'], ['Nao_pagamento','Nao_pagamento', 'Nao_pagamento',
                                                                    'Nao_pagamento','Nao_pagamento','Nao_pagamento','Nao_pagamento','Nao_pagamento','Pagamento'])
df['TARGET']  = df['TARGET'].replace(['Nao_pagamento','Pagamento' ], [0, 1])

In [ ]:
df.TARGET.value_counts()

In [ ]:
df.sample(frac=0.1).boxplot('VALOR_DIVIDA_ATIVA')

In [ ]:
df.sample(frac=0.1).boxplot('QTDE_TITULOS')

In [ ]:
df = df.query('VALOR_DIVIDA_ATIVA < 10000')
df = df.query('QTDE_TITULOS < 200 ')
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.drop(columns=['TIPO_BAIXA'], inplace=True)
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df.shape

# Pre Processamento dos dados

## Balanceamento de Variáveis

In [ ]:
target_count = df['TARGET'].value_counts()
print('Class 0:', target_count[0])
print('Class 1:', target_count[1])
print('Proportion:', round(target_count[0] / target_count[1], 2), ': 1')
target_count.plot(kind='bar', title='Count (TARGET)',color = ['#1F77B4', '#FF7F0E']);

In [ ]:
# CONTAR CLASSES
count_class_0, count_class_1 = df['TARGET'].value_counts()
# Divide by class
df_class_0 = df[df['TARGET'] == 0]
df_class_1 = df[df['TARGET'] == 1]
df_class_0_under = df_class_0.sample(count_class_1)
df = pd.concat([df_class_0_under, df_class_1], axis=0)
print('Random under-sampling:')
print(df['TARGET'].value_counts())
df['TARGET'].value_counts().plot(kind='bar', title='Count (TARGET)',color = ['#1F77B4', '#FF7F0E']);

In [ ]:
df.columns

In [ ]:
features = df[[ 'QTDE_TITULOS', 'VALOR_DIVIDA_ATIVA', 'CEP' , 'INDICADOR']] 
features_cpc = df[[ 'QTDE_TITULOS', 'VALOR_DIVIDA_ATIVA' , 'CEP', 'ORIGEM', 'TIPO']] 
target = df[['TARGET']]

## Padronização ou Normalização das variáveis

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
features['CEP']= le.fit_transform(features["CEP"])
features['INDICADOR']= le.fit_transform(features["INDICADOR"])

features_cpc['ORIGEM']= le.fit_transform(features_cpc["ORIGEM"])
features_cpc['TIPO']= le.fit_transform(features_cpc["TIPO"])
features_cpc['CEP']= le.fit_transform(features_cpc["CEP"])


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features['QTDE_TITULOS'] = scaler.fit_transform(features[['QTDE_TITULOS']])
features['VALOR_DIVIDA_ATIVA'] = scaler.fit_transform(features[['VALOR_DIVIDA_ATIVA']])

features_cpc['QTDE_TITULOS'] = scaler.fit_transform(features_cpc[['QTDE_TITULOS']])
features_cpc['VALOR_DIVIDA_ATIVA'] = scaler.fit_transform(features_cpc[['VALOR_DIVIDA_ATIVA']])

In [ ]:
del df_class_0
del df_class_1

# Modelagem preditiva

## Modelagem Preditiva com O algoritimo Devedor

In [ ]:
# Avaliação do modelo usando o XGBoost
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

inicio = time.time()

# Divide os dados em treino e teste
X = features
Y = target
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 5)

# Criando o modelo
Model = XGBClassifier(colsample_bytree=0.2, gamma=0, gpu_id=-1, learning_rate=0.2,  max_depth=6, n_estimators=150, n_jobs=48,
                         nthread=48, num_parallel_tree=1, random_state=0, subsample=0.2 ) 

# Treinando o modelo
Model.fit(X_train, Y_train)

# Previsão do modelo
Predict = Model.predict(X_test)
report = classification_report(Y_test, Predict)

# Imprimindo o relatório
print(report)

fim = time.time()
print("o tempo de execução foi de: ", fim - inicio, "segundos,  ou ",(fim - inicio)/60, "minutos" )

## Análise de Features e Acurácia

In [ ]:
# Seleção dos melhores atributos como resultado para a modelagem CPC
gc.collect()
Model.feature_importances_
feature_importances = pd.DataFrame(Model.feature_importances_,
                                   index = X.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
feature_importances



In [ ]:
# Salvando o modelo Propensao a pagar
import pickle

arquivo_cpf = '../models/modelo_v1_devedor.sav'
pickle.dump(Model, open(arquivo_cpf, 'wb'))
print("Modelo salvo!")

# Salvar as features do modelo
with open('NameFeature_v1_devedor','wb') as arquivo_cpf:
   pickle.dump(features.columns, arquivo_cpf)

print('Nomes Salvo')

# Modelagem Preditiva CPC

In [ ]:
# Avaliação do modelo usando o XGBoost
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

inicio = time.time()

# Divide os dados em treino e teste
X = features_cpc
Y = target
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 5)

# Criando o modelo
Model_cpc = XGBClassifier(colsample_bytree=0.2, gamma=0, gpu_id=-1, learning_rate=0.2,  max_depth=6, n_estimators=150, n_jobs=48,
                         nthread=48, num_parallel_tree=1, random_state=0, subsample=0.2 ) 

# Treinando o modelo
Model_cpc.fit(X_train, Y_train)

# Previsão do modelo
Predict = Model_cpc.predict(X_test)
report = classification_report(Y_test, Predict)

# Imprimindo o relatório
print(report)

fim = time.time()
print("o tempo de execução foi de: ", fim - inicio, "segundos,  ou ",(fim - inicio)/60, "minutos" )

In [ ]:
# Seleção dos melhores atributos como resultado para a modelagem CPC
gc.collect()
Model_cpc.feature_importances_
feature_importances = pd.DataFrame(Model_cpc.feature_importances_,
                                   index = X.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
feature_importances

In [ ]:
# Salvando o modelo CPC
import pickle

arquivo_cpc = '../models/modelo_v1_cpc.sav'
pickle.dump(Model_cpc, open(arquivo_cpc, 'wb'))
print("Modelo salvo!")

# Salvar as features do modelo
with open('NameFeature_v1_cpc','wb') as arquivo_cpc:
   pickle.dump(features_cpc.columns, arquivo_cpc)

print('Nomes Salvo')

# Deploy do modelo